# Julia-Notebook zur Veranschaulichung der Pol-Nullstellen-Methode

## Initialisierung

In [1]:
using Interact, Plots, DSP, LaTeXStrings

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9195445619013469550\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-892092328889834619\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9195445619013469550\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Nützliche Funktionen:

In [34]:
function circle(r)
    theta = LinRange(0, 2*pi, 500)
    sin.(theta), cos.(theta)
end

function zplane(z, p)
    plot(circle(1), seriestype = [:shape,], c = :black, fillalpha = .2, aspect_ratio = 1,
        label=false, legend=:bottomright)
    scatter!(real(z), imag(z), c = :blue, label="Nullstellen")
    scatter!(real(p), imag(p), c = :red, label="Polstellen")
end

zplane (generic function with 1 method)

## Bandpass

Parameter:
* Abtastfrequenz $f_s$
* Bandbreite $B$
* Mittenfrequenz $f_0$
* ein gegebener Wert von $H^\prime(f)=H\left(e^{j2\pi f/ f_s}\right)$, z.B., $H^\prime(f_0)=1$

Nullstellen: $z_{0,1}=1$ und $z_{0,2}=-1$

Polstellen: $z_{\infty,1/2}=r e^{\pm j 2 \pi \frac{f_0}{f_s}}$ mit $r=1-\frac{B}{f_s}\pi$

Funktion zur Ermittlung der Filterkoeffzienten (Annahme: $H^\prime(f_0)=1$):

In [24]:
function pole_zero_method_BP(f_0_norm, B_norm)
    # Nullstellen
    z = [1, -1]

    # Pole
    r = 1 - B_norm*pi
    z_0 = exp(im*2*pi*f_0_norm)
    p = [r*z_0, r*conj(z_0)]

    # Berechnung des Koeffizienten b_0 (gain k)
    k = (z_0-p[1])*(z_0-p[2])/(z_0-z[1])/(z_0-z[2])

    # Filterkoeffizienten
    filter = ZeroPoleGain(z, p, k)
    
    return z, p, filter
    
end

pole_zero_method_BP (generic function with 1 method)

Resultierender Frequenzgang:

In [35]:
@manipulate for f_0_norm in 0:.01:.5, B_norm in 0.01:.01:.5
    f_norm = 0:.001:.5
    z, p, filter = pole_zero_method_BP(f_0_norm, B_norm)
    H = freqz(filter, f_norm*2*pi)
    a = plot(f_norm, 20*log10.(abs.(H)), label=false, lw=2, linecolor = :blue, legend=:bottomright)
    vline!([f_0_norm-B_norm/2], label = L"f = f_0-B/2", linecolor = :green)
    vline!([f_0_norm+B_norm/2], label = L"f = f_0+B/2", linecolor = :green)
    xlims!((0,.5))
    ylims!((-50,5))
    xlabel!(L"f/f_s")
    ylabel!(L"20 \log_{10}H^\prime(f)")
    b = zplane(z, p)
    plot(a, b)
  end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["f_0_norm"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 51,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
26, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/home/guido/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/guido/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/guido/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/home/guido/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/home/guido/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007fa9af138eb0, Task (runnable) @0x00007fa9af138eb0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\"

## Bandsperre

Parameter:
* Abtastfrequenz $f_s$
* Bandbreite $B$
* Mittenfrequenz $f_0$
* ein gegebener Wert von $H^\prime(f)=H\left(e^{j2\pi f/ f_s}\right)$, z.B., $H^\prime(0)=1$

Nullstellen: $z_{0,1/2}=e^{\pm j 2 \pi \frac{f_0}{f_s}}$

Polstellen: $z_{\infty,1/2}=r e^{\pm j 2 \pi \frac{f_0}{f_s}}$ mit $r=1-\frac{B}{f_s}\pi$

Funktion zur Ermittlung der Filterkoeffzienten (Annahme: $H^\prime(0)=1$):

In [36]:
function pole_zero_method_BS(f_0_norm, B_norm)
    # Nullstellen
    z_0 = exp(im*2*pi*f_0_norm)
    z = [z_0, conj(z_0)]

    # Pole
    r = 1 - B_norm*pi
    p = [r*z_0, r*conj(z_0)]

    # Berechnung des Koeffizienten b_0 (gain k)
    k = (1-p[1])*(1-p[2])/(1-z[1])/(1-z[2])

    # Filterkoeffizienten
    filter = ZeroPoleGain(z, p, k)
    return z, p, filter
    
end;

Resultierender Frequenzgang:

In [38]:
@manipulate for f_0_norm in 0:.01:.5, B_norm in 0.01:.01:.5
    f_norm = 0:.001:.5
    z, p, filter = pole_zero_method_BS(f_0_norm, B_norm)
    H = freqz(filter, f_norm*2*pi);
    a = plot(f_norm, 20*log10.(abs.(H)), label=false, lw=2, linecolor = :blue)
    vline!([f_0_norm-B_norm/2], label = L"f = f_0-B/2", linecolor = :green)
    vline!([f_0_norm+B_norm/2], label = L"f = f_0+B/2", linecolor = :green)
    xlims!((0,.5))
    ylims!((-50,5))
    xlabel!(L"f/f_s")
    ylabel!(L"20 \log_{10}H^\prime(f)")
    b = zplane(z, p)
    plot(a,b)
  end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["f_0_norm"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 51,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
26, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/home/guido/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/guido/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/guido/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/home/guido/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/home/guido/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007fa9af870010, Task (runnable) @0x00007fa9af870010), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\"